<a href="https://colab.research.google.com/github/rajatdeveloper/NSEP-Data-Analytics/blob/master/Part1_%26_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing &  Importing Dataset and Packages

# Data Preprocessing

**Import Dataset**

In [0]:
pip install nsepy

**Import Packages and Library**

In [0]:
from datetime import date,timedelta
from nsepy import get_history
import numpy as np
import pandas as pd
from pandas import datetime
import matplotlib.pyplot as plt
import math
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 12)

**NSE stocks (INFY,TCS) between 2015-2016**

In [0]:
number_of_days = 365
#start date
strt = date(2015,1,1)
#end date
ends = strt+timedelta(days=number_of_days)
#fetches the price history of stocks
TCS = get_history(symbol='TCS',
                    start= strt,
                    end=ends)



In [0]:
TCS

In [0]:
number_of_days = 365
#start date
strt = date(2015,1,1)
#end date
ends = strt+timedelta(days=number_of_days)
#fetches the price history of stocks
INFY = get_history(symbol='INFY',
                    start= strt,
                    end=ends)

In [0]:
INFY

**Type Casting Date**

In [0]:
def type_casting(stock):
    #reset the index
    stock = stock.reset_index()
    #typecast the Date datatype    
    stock["Date"] = pd.to_datetime(stock["Date"])
    #assign the index
    stock.index = stock['Date']
    return stock

In [0]:
TCS = type_casting(TCS)
INFY = type_casting(INFY)
#Assign name to stocks
TCS.name = 'TCS'
INFY.name = 'INFY'

In [0]:
#create list of stock
stocks = [TCS,INFY]

#                                                                                             Part 1

**1. Create 4,16,....,52 week moving average(closing price) for each stock and index. This should happen through a function.**

Assumption : Assume the size of a week is 5 day (Monday-Friday) and ignore the other holidays.
Can't compute 52-week moving avergae because of other holidays

**Moving Average Implementation**

In [0]:
#Moving Avergae Function
def moving_average(values, size):
    return np.convolve(values, np.ones(size), 'valid')/ size

In [0]:
#Append Moving Average in Dataset & Plot Line Chart Of Moving Average
def plot_moving_average(stock):
  
  #creating array of window size
  window_arr = [4,16,28,40]
  day = 5
  moving_average_dict = {}
  
  #Plot of closing price of stock
  plt.title("Moving Average Of "+stock.name+" Stock",fontsize=20)
  plt.xlabel("Date",fontsize=15) 
  plt.ylabel("Price",fontsize=20)
  #Plot of original closing price of stock
  plt.plot(stock["Date"],stock["Close"],label="Closing pricing",linestyle='--',linewidth=2)
  
  for size in window_arr:
   
    #dummy array for starting date to resize the frame as original 
    NA = [float('nan') for i in range(size*day-1)]
    NA = np.array(NA)
    #append dummy array with moving average
    moving_average_dict[size] = np.hstack([NA,moving_average(stock["Close"],size*day)])
   
    #append moving average to original dataframe of stock
    temp = "Moving Average "+str(size)+"-week"
    stock[temp] = moving_average_dict[size]
    
    #Plot of moving average of closing price of stock
    plt.plot(stock["Date"],stock[temp],label = temp,linewidth=2)
  
  print("Data Of "+stock.name)
  #display stock dataframe
  display(stock)
  
  plt.legend(title = "Legends",loc = 3,prop={'size': 12})
  plt.show()

In [0]:
for stock in stocks:
  plot_moving_average(stock)


**2. Create rolling window of size 10 on each stock/index. Handle unequal time series due to stock market holidays. You should look to increase your rolling window size to 75 and see how the data looks like.**

In [0]:
def rolling_window(stock):
    #creating array of Rolling window size
    rolling_arr = [10,75]
    
    #Plot of closing price of stock
    plt.title("Moving Average Of "+stock.name+" Stock",fontsize=20)
    plt.xlabel("Date",fontsize=15) 
    plt.ylabel("Price",fontsize=20)  
    #Plot of original closing price of stock
    plt.plot(stock["Date"],stock["Close"],label="Closing pricing",linestyle='--',linewidth=2)

    for size in rolling_arr:
        temp = "Rolling Window "+str(size) 
        
        #append rolling window average to original dataframe of stock
        stock[temp] = np.round(stock["Close"].rolling(window = size, center = False).mean(), 2)
        
        #Plot of rolling window average of closing price of stock
        plt.plot(stock["Date"],stock[temp],label = temp,linewidth=2)
    
    print("Data Of "+stock.name)
    #display stock dataframe
    display(stock)
    
    plt.legend(title = "Legends",loc = 3,prop={'size': 12})
    plt.show()

In [0]:
for stock in stocks:
  rolling_window(stock)


**3.1 Volume shocks - If volume traded is 10% higher/lower than previous day - make a 0/1 boolean time series for shock, 0/1 dummy-coded time series for direction of shock.**

In [0]:
#Previous Day Volume
def previous_volume(stock):
  stock["Previous Day Volume"] = stock["Volume"].shift(1)


#Volume Shock
def volume_shocks(stock):
  stock["Volume Shock"] = ((((abs(stock["Previous Day Volume"]-stock["Volume"]))/stock["Volume"])*100)>10).astype(int)

#Volume Shock Direction    
def volume_shock_direction(stock):
  if(stock["Volume Shock"]==1):
        #if volume is greater than previous day direction is 1 else 0
        if(stock["Volume"]-stock["Previous Day Volume"]>0):
            return 1
        else:
            return 0
  else:
        return float('nan')
  

In [0]:
for stock in stocks:
  previous_volume(stock)
  volume_shocks(stock)
  stock["Volume Shock Direction"] = stock.apply(volume_shock_direction,axis=1)
  #display stock dataframe
  print("Data Of "+stock.name)
  display(stock)
  


**3.2 Price shocks  & Pricing black swan - If closing price at T vs T+1 has a difference > 2%, then 0/1 boolean time series for shock, 0/1 dummy-coded time series for direction of shock
(3.2 & 3.3 both are same)**

In [0]:
#Next Day Close
def next_close(stock):
  stock["Next Day Close"] = stock["Close"].shift(-1)


#Price Shock
def price_shocks(stock):
  stock["Price Shock"] = ((((abs(stock["Next Day Close"]-stock["Close"]))/stock["Close"])*100)>2).astype(int)

#Price Shock Direction    
def price_shock_direction(stock):
  if(stock["Price Shock"]==1):
        #if close is greater than next day direction is 1 else 0
        if(stock["Close"]-stock["Next Day Close"]>0):
            return 1
        else:
            return 0
  else:
        return float('nan')
  

In [0]:
for stock in stocks:
  next_close(stock)
  price_shocks(stock)
  stock["Price Shock Direction"] = stock.apply(price_shock_direction,axis=1)
  #display stock dataframe
  print("Data Of "+stock.name)
  display(stock)
  


**3.4 Pricing shock without volume shock - based on points a & b - Make a 0/1 dummy time series.
 **

In [0]:
def pshock_without_vshock(stock):  
  stock["Pshock W/O Vshock"] = ((~stock["Volume Shock"]) & stock["Price Shock"]).astype(int)

In [0]:
for stock in stocks:
  pshock_without_vshock(stock)
  #display stock dataframe
  print("Data Of "+stock.name)
  display(stock)
  


#                                                                                             Part 2

**Import Plotting Packages and Library**

In [0]:
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer,GnBu,Blues
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource,LinearColorMapper,BasicTicker, PrintfTickFormatter, ColorBar
from statsmodels.tsa.stattools import acf, pacf

In [0]:
output_notebook()

In [0]:
def bokeh_visual(stock):
    print(stock.name)
    fig = figure(x_axis_type="datetime")
    
    #Color timeseries in simple blue color
    fig.line(stock['Date'], stock['Close'], color='blue', alpha=0.5)
    
    #Color timeseries between two volume shocks in a different color (Red)
    flag = False
    last = 0
    segments = []
    for i in range(len(stock["Volume Shock"])):
        if(stock["Volume Shock"][i] and flag):
                fig.line(stock['Date'][last:i], stock['Close'][last:i], color='red', alpha=0.5)
                segments.append((last,i))
                flag = False
        elif(stock["Volume Shock"][i]):
                last = i
                flag = True
              
    #Mark closing Pricing shock without volume shock to identify volumeless price movement
    fig.circle(stock.index, stock.Close*stock["Pshock W/O Vshock"], size=4, legend='Price Shock W/O Volume Shock')
    show(fig)

In [0]:
for stock in stocks:
    bokeh_visual(stock)

In [0]:
#Hand craft partial autocorrelation plot for each stock/index on upto all lookbacks on bokeh
def plot_pacf(stock):
    print(stock.name)
    lags = 50
    x = list(range(lags))
    fig = figure(plot_height=500, title="Partial Autocorrelation Plot " +stock.name)
    partial_autocorr = pacf(stock["Close"], nlags=lags-1)
    fig.vbar(x=x, top=partial_autocorr, width=0.9)
    show(fig)

In [26]:
for stock in stocks:
    plot_pacf(stock)

TCS


INFY
